In [5]:
!pip install accelerate -U



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/302.4 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 194.6/302.4 kB 5.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.9 MB/s eta 0:00:00















  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)


  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)


  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)


  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)


  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)


  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)


  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)


  Using cached nvidia_cusparse_cu12-

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import Trainer, TrainingArguments

2024-05-07 20:43:08.311980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 20:43:08.312084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 20:43:08.436389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df1 = pd.read_csv('/kaggle/input/intents/Intent0_final.csv')
df2 = pd.read_csv('/kaggle/input/intents/Intent1_final.csv')

In [7]:
df = pd.concat([df1,df2], ignore_index=True)

In [8]:
df.shape

(120520, 2)

In [9]:
df['Label'].value_counts()

Label
0    82886
1    37634
Name: count, dtype: int64

In [10]:
df.dropna(subset=['utterance','Label'], inplace=True)

In [11]:
df['utterance'] = df['utterance'].astype(str)
df['Label'] = df['Label'].astype(int)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# df.to_csv('final_classification_data.csv')

In [12]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
training_encodings = tokenizer(train_df['utterance'].tolist(), truncation=True, padding=True, max_length=128)
validation_encodings = tokenizer(val_df['utterance'].tolist(), truncation=True, padding=True, max_length=128)

In [13]:
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

training_dataset = IntentDataset(training_encodings, train_df['Label'].values)
validation_dataset = IntentDataset(validation_encodings, val_df['Label'].values)

In [14]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=300,
    report_to='none'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
300,0.177300
600,0.037900
900,0.027800
1200,0.023100
1500,0.023000
1800,0.019700
2100,0.019100
2400,0.017100
2700,0.017000
3000,0.011400


TrainOutput(global_step=6780, training_loss=0.019641809486358214, metrics={'train_runtime': 2537.2962, 'train_samples_per_second': 85.499, 'train_steps_per_second': 2.672, 'total_flos': 1.426956497639424e+16, 'train_loss': 0.019641809486358214, 'epoch': 2.0})

In [15]:
model_path = "/content/model"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

In [16]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.0014644584152847528, 'eval_runtime': 389.8882, 'eval_samples_per_second': 278.203, 'eval_steps_per_second': 8.695, 'epoch': 2.0}


In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [18]:
model.push_to_hub("nlpproject/IntentClassification_01")
tokenizer.push_to_hub("nlpproject/IntentClassification_01")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nlpproject/IntentClassification_01/commit/c1d263e80f468d8f0613ab8c26a530612d33cf4a', commit_message='Upload tokenizer', commit_description='', oid='c1d263e80f468d8f0613ab8c26a530612d33cf4a', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
test_texts = ["what are top 5 oceans", 'i am sad']
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

In [20]:
model.eval()
device = torch.device("cpu")
model = model.to(device)
with torch.no_grad():
    outputs = model(**{k: v.to(device) for k, v in test_encodings.items()})
    logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)
predictions = torch.argmax(probabilities, dim=1).cpu().numpy()

print("Predictions:", predictions)

Predictions: [1 0]


**Evaluation**

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlpproject/IntentClassification_01"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [4]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [14]:
test_df1 = test_df.sample(500)  # notebook is crashing if we test with more samples

In [15]:
test_df1 = test_df1[['utterance', 'Label']]

In [16]:
test_texts = test_df1['utterance'].tolist()
test_labels = test_df1['Label'].tolist()

In [17]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [18]:
test_inputs = {
    'input_ids': torch.tensor(test_encodings['input_ids']),
    'attention_mask': torch.tensor(test_encodings['attention_mask'])
}

In [19]:
with torch.no_grad():
    outputs = model(**test_inputs)
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

In [45]:
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='weighted')
recall = recall_score(test_labels, predicted_labels, average='weighted')
f1 = f1_score(test_labels, predicted_labels, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.838
Precision: 0.8465294630195964
Recall: 0.838
F1 Score: 0.8404371221998342
